In [1]:
import os, glob
import torch
import pandas as pd
from PIL import Image
from tqdm import tqdm
from torchvision import transforms
from HGNN.train import CNN, dataLoader
from HGNN.train.configParser import ConfigParser

def MakeSquared(img,imageDimension = 224):
        img_H = img.size[0]
        img_W = img.size[1]

        # Resize
        smaller_dimension = 0 if img_H < img_W else 1
        larger_dimension = 1 if img_H < img_W else 0
        new_smaller_dimension = int(imageDimension * img.size[smaller_dimension] / img.size[larger_dimension])
        if smaller_dimension == 1:
            img = transforms.functional.resize(img, (new_smaller_dimension, imageDimension))
        else:
            img = transforms.functional.resize(img, (imageDimension, new_smaller_dimension))

        # pad
        diff = imageDimension - new_smaller_dimension
        pad_1 = int(diff/2)
        pad_2 = diff - pad_1
        RGBmean = [0.485*255, 0.456*255, 0.406*255]
        fill = tuple([round(x) for x in RGBmean])

        if smaller_dimension == 0:
            img = transforms.functional.pad(img, (pad_1, 0, pad_2, 0), padding_mode='constant', fill = fill)
        else:
            img = transforms.functional.pad(img, (0, pad_1, 0, pad_2), padding_mode='constant', fill = fill)

        return img
    
def getTransforms(normalize=False):
    transformsList = [transforms.Lambda(MakeSquared),transforms.ToTensor()]
    normalizer = [transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])]
    if normalize:
        transformsList = transformsList + normalizer
    return transformsList

def image_loader(image_name):
    """load image, returns cuda tensor"""
    image = Image.open(image_name)
    loader = transforms.Compose(getTransforms())
    image = loader(image).float()
    #image = Variable(image, requires_grad=True)
    #image = image.unsqueeze(0)  #this is for VGG, may not be needed for ResNet
    return image.cuda()  #assumes that you're using GPU



In [9]:
experiments_df["trialHash"][0]


'c1993d5dd74c14a6bf78424f55c1d81e4e7b22d6ee5f73e6efee7ce9'

In [10]:
experimentsPath="/home/jnl47/Documents/dev/HGNN/experiments/"
dataPath="/home/jnl47/Documents/dev/HGNN/data"
experimentName="smegma"
trial_hash="c1993d5dd74c14a6bf78424f55c1d81e4e7b22d6ee5f73e6efee7ce9"

experimentsFileName = "experiments.csv"
#Get experiment parameters
#can I not getthese from params?
experimentsFileNameAndPath = os.path.join(experimentsPath, experimentsFileName)
if os.path.exists(experimentsFileNameAndPath):
    experiments_df = pd.read_csv(experimentsFileNameAndPath)
else:
    raise Exceptions("Can't find {}".format(experimentsFileNameAndPath))
if trial_hash not in experiments_df["trialHash"]:
    raise Exception("Experiment not " + experiments_df["trialHash"] + " found!")
experimentRecord = experiments_df[experiments_df["trialHash"] == trial_hash]
experiment_params = experimentRecord.to_dict('records')[0]


Exception: 0     Experiment not c1993d5dd74c14a6bf78424f55c1d81...
1     Experiment not 1f0aba55b3312a4eee7d0ff88ba7164...
2     Experiment not 06feff02666707cbdecf12fdbec650a...
3     Experiment not 2bd5d4eb14dcb59ec9c21d680871670...
4     Experiment not 53b48db3fa0b2dc3f440b629999b963...
5     Experiment not b4c0f0143d3ab8563afe9d950ff4200...
6     Experiment not 48a295586a5ad67e7c7eae8e640dd23...
7     Experiment not e4d2061ecb3aa4b754db35f94628d44...
8     Experiment not facf6012c5f10a7d67488303b8f3fef...
9     Experiment not 1c3884e98bdeb97505c2aedd9b9b7c2...
10    Experiment not 2bcce9eb2c4048599403606403e7264...
11    Experiment not 0ad8eb7c2d79b9f282677e32cc54761...
12    Experiment not e310ae2705796efd3f282d214a47ec6...
13    Experiment not d19a29e954bc6558e8808a2c5402b95...
14    Experiment not 39f6cc789b1a2b3581fd05b0c7e0cc3...
15    Experiment not fa5fabc92d55a20b96a4abf53fec8aa...
16    Experiment not ff4a83095fc77b0f4209983c2dff60e...
17    Experiment not 2702d15c289f2bfa

In [4]:
trialHash in experiments_df.trialHash.values

NameError: name 'trialHash' is not defined

In [5]:
config_parser = ConfigParser(experimentsPath, dataPath, experimentName)
experiment_params = config_parser.read()
experimentPathAndName = os.path.join(experimentsPath, experimentName)
datasetManager = dataLoader.datasetManager(experimentPathAndName)
datasetManager.updateParams(config_parser.fixPaths(experiment_params))
dataset = datasetManager.getDataset()


AttributeError: 'ConfigParser' object has no attribute 'read'

In [ ]:
train_loader, validation_loader, test_loader = datasetManager.getLoaders()

In [11]:
experimentsFileNameAndPath = os.path.join(experimentsPath, experimentsFileName)
if os.path.exists(experimentsFileNameAndPath):
    experiments_df = pd.read_csv(experimentsFileNameAndPath)
experiments_df["trialHash"]
experiments_df[experiments_df["trialHash"] == trial_hash]

architecture = {
    "fine": len(dataset.csv_processor.getFineList()),
    "coarse" : len(dataset.csv_processor.getCoarseList())
}
dataset.csv_processor.getFineList()

['Lepomis auritus',
 'Noturus eleutherus',
 'Carassius auratus',
 'Lepomis cyanellus',
 'Esox americanus',
 'Noturus insignis',
 'Noturus flavus',
 'Noturus gyrinus',
 'Esox lucius',
 'Noturus exilis',
 'Cyprinus carpio']

In [39]:
model = CNN.create_model(architecture, experiment_params)


In [40]:
model


CNN_hierarchy(
  (pretrained_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [41]:
# get the model and the parameters
modelName = experimentRecord.iloc[0]["modelName"]
trialName = os.path.join(experimentPathAndName, modelName)
CNN.loadModel(model, trialName)

(      Unnamed: 0  validation_fine_f1  training_fine_f1  test_fine_f1  \
 0              0            0.107183          0.118520      0.102010   
 1              1            0.220853          0.304469      0.167033   
 2              2            0.336211          0.454915      0.262116   
 3              3            0.361570          0.587013      0.374590   
 4              4            0.428658          0.686174      0.446812   
 ...          ...                 ...               ...           ...   
 1170        1170            0.767269          1.000000      0.807946   
 1171        1171            0.751419          1.000000      0.800982   
 1172        1172            0.779072          1.000000      0.819837   
 1173        1173            0.779072          1.000000      0.810383   
 1174        1174            0.767269          1.000000      0.810026   
 
       validation_loss  training_loss  training_coarse_loss  \
 0            2.393914       2.390469              1.609621

In [25]:
model

CNN_hierarchy(
  (pretrained_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [19]:
datasetManager.updateParams(config_parser.fixPaths(experiment_params))
dataset = datasetManager.getDataset()

# For analyzing experiments, we don't care about augmentation
dataset.toggle_image_loading(augmentation=False, normalization=dataset.normalization_enabled)

train_loader, validation_loader, test_loader = datasetManager.getLoaders()
fineList = dataset.csv_processor.getFineList()
coarseList = dataset.csv_processor.getCoarseList()
numberOffine = len(fineList)
numberOfcoarse = len(coarseList)
architecture = {
    "fine": numberOffine,
    "coarse" : numberOfcoarse
}
        
#model.load_state_dict(torch.load(os.path.join(savedModelName, modelFinalCheckpoint))) 
#model.eval()
#ptModelAxial.load_state_dict(dTemp['state_dict'])
#for p in ptModelAxial.parameters():
#    p.requires_grad = False
#ptModelAxial.eval()

#InputImg = skimage.img_as_float(skimage.io.imread(sFileName))
#ptModelPreds = ptModelAxial( Torchify(InputImg) )
#print( ptModelPreds )


#mymodel=torch.load(os.path.join(experimentsPath,experimentName,"models",trial_hash,"finalModel.pt"))

Creating loaders...
Creating loaders... Done.


In [17]:
df, epochs, time_elapsed = CNN.loadModel(os.path.join(experimentsPath,experimentName), trial_hash)

AttributeError: 'str' object has no attribute 'load_state_dict'

In [54]:
model.load_state_dict(torch.load("../../experiments/smegma/models/06feff02666707cbdecf12fdbec650a36bef56a47085a37a2ddf1515/finalModel.pt"))
_ = model.eval()

In [77]:
# https://stackoverflow.com/q/57237381/264696
preds=model(lepomis_auritus[None, ...])

In [80]:
probs=torch.nn.Softmax(dim=1)(preds['fine'])
probs

tensor([[0.0760, 0.5243, 0.0854, 0.0296, 0.0296, 0.0296, 0.0296, 0.0547, 0.0296,
         0.0296, 0.0822]], device='cuda:0', grad_fn=<SoftmaxBackward>)

In [79]:
preds['fine'].topk(1)

torch.return_types.topk(
values=tensor([[2.8752]], device='cuda:0', grad_fn=<TopkBackward>),
indices=tensor([[1]], device='cuda:0'))

In [84]:
preds=model(noturus_eleutherus[None, ...])

In [85]:
probs=torch.nn.Softmax(dim=1)(preds['fine'])
probs

tensor([[0.0443, 0.2424, 0.1522, 0.0338, 0.0338, 0.0338, 0.0743, 0.0338, 0.0760,
         0.2418, 0.0338]], device='cuda:0', grad_fn=<SoftmaxBackward>)

In [87]:
probs=torch.nn.Softmax(dim=1)(preds['coarse'])
probs

tensor([[0.0627, 0.4638, 0.2379, 0.1728, 0.0627]], device='cuda:0',
       grad_fn=<SoftmaxBackward>)

In [90]:
preds=model(esox_americanus[None, ...])

In [91]:
probs=torch.nn.Softmax(dim=1)(preds['fine'])
probs

tensor([[0.0188, 0.0112, 0.0497, 0.0222, 0.0112, 0.0112, 0.0654, 0.0167, 0.7470,
         0.0355, 0.0112]], device='cuda:0', grad_fn=<SoftmaxBackward>)

In [88]:
dataset.csv_processor.getCoarseList()

['Lepomis', 'Noturus', 'Carassius', 'Esox', 'Cyprinus']

In [73]:
dataset.csv_processor.get_image_full_path()

'/home/jnl47/Documents/dev/HGNN/data/INHS_segmented_padded_fish/images'

In [81]:
dataset.csv_processor.getFineList()

['Lepomis auritus',
 'Noturus eleutherus',
 'Carassius auratus',
 'Lepomis cyanellus',
 'Esox americanus',
 'Noturus insignis',
 'Noturus flavus',
 'Noturus gyrinus',
 'Esox lucius',
 'Noturus exilis',
 'Cyprinus carpio']

In [82]:
dataset.csv_processor.samples

[{'fine': 'Lepomis auritus',
  'coarse': 'Lepomis',
  'fileName': 'INHS_FISH_68432.jpg',
  'images': [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3243x1663 at 0x7F9D08403FD0>]},
 {'fine': 'Noturus eleutherus',
  'coarse': 'Noturus',
  'fileName': 'INHS_FISH_2426.jpg',
  'images': [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2491x1004 at 0x7F9D08403F50>]},
 {'fine': 'Carassius auratus',
  'coarse': 'Carassius',
  'fileName': 'INHS_FISH_93450.jpg',
  'images': [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2091x847 at 0x7F9D08620090>]},
 {'fine': 'Lepomis cyanellus',
  'coarse': 'Lepomis',
  'fileName': 'INHS_FISH_89331.jpg',
  'images': [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2562x1208 at 0x7F9E20873710>]},
 {'fine': 'Esox americanus',
  'coarse': 'Esox',
  'fileName': 'INHS_FISH_95243.jpg',
  'images': [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3972x1098 at 0x7F9E240DFC90>]},
 {'fine': 'Noturus eleutherus',
  'coarse': 'Noturu